<a href="https://colab.research.google.com/github/botatooo/pp-detection-fracture-recherche/blob/dev/src/fracatlas_efficientdet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
import json
from torchvision.datasets.utils import download_and_extract_archive

root = "dataset/"
url = "https://figshare.com/ndownloader/files/41725659"
filename = "fracatlas.zip"

# if download:
if not os.path.isdir(os.path.join(root, "FracAtlas")):
    os.makedirs(root, exist_ok=True)
    download_and_extract_archive(
        url,
        os.path.dirname(root),
        filename=filename,
        remove_finished=True,
    )
if not os.path.isdir(root):
    raise RuntimeError(
        "Dataset not found or corrupted. You can use download=True to download it"
    )

with open("dataset/FracAtlas/Annotations/COCO JSON/COCO_fracture_masks.json") as f:
  fracture_masks_data = json.load(f)

fractured_images = [i["file_name"] for i in fracture_masks_data["images"]]
fractured_image_count = len(fractured_images)

training_images = fractured_images[: int(0.9 * fractured_image_count)]
testing_images = fractured_images[int(0.9 * fractured_image_count) :]


os.mkdir("data")
os.mkdir("data/fracatlas")


os.mkdir("data/fracatlas/images")

os.mkdir("data/fracatlas/images/train")
for i in training_images:
  full_path = os.path.abspath(os.path.join("dataset/FracAtlas/images/Fractured", i))
  new_path = os.path.abspath(os.path.join("data/fracatlas/images/train", i))
  os.rename(full_path, new_path)

os.mkdir("data/fracatlas/images/val")
for i in testing_images:
  full_path = os.path.abspath(os.path.join("dataset/FracAtlas/images/Fractured", i))
  new_path = os.path.abspath(os.path.join("data/fracatlas/images/val", i))
  os.rename(full_path, new_path)


os.mkdir("data/fracatlas/labels")

os.mkdir("data/fracatlas/labels/train")
for i in training_images:
  i = i.replace(".jpg", ".txt")
  full_path = os.path.abspath(os.path.join("dataset/FracAtlas/Annotations/YOLO", i))
  new_path = os.path.abspath(os.path.join("data/fracatlas/labels/train", i))
  os.rename(full_path, new_path)

os.mkdir("data/fracatlas/labels/val")
for i in testing_images:
  i = i.replace(".jpg", ".txt")
  full_path = os.path.abspath(os.path.join("dataset/FracAtlas/Annotations/YOLO", i))
  new_path = os.path.abspath(os.path.join("data/fracatlas/labels/val", i))
  os.rename(full_path, new_path)


100%|██████████| 338412751/338412751 [00:12<00:00, 26915354.00it/s]


Extracting dataset/fracatlas.zip to dataset


In [8]:
!python -m pip install 'git+https://github.com/facebookresearch/detectron2.git'


  Cloning https://github.com/facebookresearch/detectron2.git to /tmp/pip-req-build-as2m_ngq
  Running command git clone --filter=blob:none --quiet https://github.com/facebookresearch/detectron2.git /tmp/pip-req-build-as2m_ngq
  Resolved https://github.com/facebookresearch/detectron2.git to commit 864913f0e57e87a75c8cc0c7d79ecbd774fc669b
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.2/50.2 kB 856.5 kB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.5/79.5 kB 3.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.5/154.5 kB 6.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 15.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.0/117.0 kB 9.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for detectron2: filename=detectron2-0.6-cp310-cp310-linux_x86_64.whl size=5739958 sha256=b3486926fc667bad972a0bc9e149

In [9]:
# !git clone https://github.com/mtroym/EfficientDet.detectron2
%cd "EfficientDet.detectron2"
!DETECTRON2_DATASETS=../data/ python3 train.py --config-file configs/Base-EfficientDet-COCO.yaml

[Errno 2] No such file or directory: 'EfficientDet.detectron2'
/content/EfficientDet.detectron2
Command Line Args: Namespace(config_file='configs/Base-EfficientDet-COCO.yaml', resume=False, eval_only=False, num_gpus=1, num_machines=1, machine_rank=0, dist_url='tcp://127.0.0.1:49152', opts=[])
Loading config configs/Base-EfficientDet-COCO.yaml with yaml.unsafe_load. Your machine may be at risk if the file contains malicious content.
[01/09 05:32:11 detectron2]: Rank of current process: 0. World size: 1
No CUDA runtime is found, using CUDA_HOME='/usr/local/cuda'
[01/09 05:32:12 detectron2]: Environment info:
-------------------------------  -----------------------------------------------------------------
sys.platform                     linux
Python                           3.10.12 (main, Nov 20 2023, 15:14:05) [GCC 11.4.0]
numpy                            1.23.5
detectron2                       0.6 @/usr/local/lib/python3.10/dist-packages/detectron2
Compiler                         GC